In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 28.3 MB/s eta 0:00:00


In [3]:
import pandas as pd

In [4]:
pollution = pd.read_csv('/content/drive/MyDrive/Papers/dynamic_programming/data/NYC_Pilot1_PM.csv')

In [5]:
pollution.head(2)

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm1,pm25,pm10
0,NYCP1_01A,1579618560,40.847183,-73.870087,23,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.3,15.2,1.44,5.91,11.35
1,NYCP1_01A,1579618560,40.847183,-73.870094,18,2,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,16.2,15.1,1.05,1.18,1.18


In [6]:
len(pollution)

118765

A dictionary called vars is created which contains the LP variables. The reference keys to the dictionary are the warehouse name, then the bar name([“A”][“2”]) , and the data is Route_Tuple. (e.g. [“A”][“2”]: Route_A_2). The lower limit of zero is set, the upper limit of None is set, and the variables are defined to be Integers.



```
A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)
```

[pulp](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html)

In [11]:
import pandas as pd
import pulp
from math import radians, sin, cos, sqrt, atan2

def distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two points using Haversine formula
    R = 6371.0  # Radius of the Earth in kilometers
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance
problem =None
def minimum_cost_deployment(csv_file, budget, coverage_threshold, connectivity_threshold):
    # Read pollution data from CSV file
    data = pd.read_csv(csv_file)

    # Initialize the ILP problem
    problem = pulp.LpProblem("MinimumCostDeployment", pulp.LpMinimize)

    # Decision variables
    num_sites = len(data)
    sensors = pulp.LpVariable.dicts("Sensor", range(num_sites), cat=pulp.LpBinary)

    # Objective function: minimize cost
    problem += pulp.lpSum([sensors[i] for i in range(num_sites)])

    # Constraints: pollution coverage
    for i in range(num_sites):
        problem += pulp.lpSum([sensors[j] for j in range(num_sites) if distance(data['latitude'][i], data['longitude'][i], data['latitude'][j], data['longitude'][j]) <= coverage_threshold]) >= 1

    # Constraint: budget constraint
    problem += pulp.lpSum([sensors[i] for i in range(num_sites)]) <= budget

    # Solve the problem
    problem.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[problem.status])
    # Extract solution
    solution = []
    for i in range(num_sites):
        if sensors[i].varValue == 1:
            solution.append(i)

    return solution



In [9]:
from pulp import *

In [12]:

# Example usage:
csv_file = "/content/drive/MyDrive/Papers/dynamic_programming/data/data.csv"  # Replace with the path to your CSV file
budget = 5  # Example budget
coverage_threshold = 0.1  # Example coverage threshold in kilometers
connectivity_threshold = 10  # Example connectivity threshold in kilometers

# Call the function
solution = minimum_cost_deployment(csv_file, budget, coverage_threshold, connectivity_threshold)
print("Optimal sensor deployment:", solution)

Status: Optimal
Optimal sensor deployment: [179, 200, 494]
